# Task 2 find negative values

Given a list of integer numbers, look for a negative number in the list. Consider an appropriate number of qubits and explain why your proposal is valid for all kinds of numbers in case 


def find_negative_numbers(list[int]:list_number):
     “””
list_number : integer list!.
Return the True or False depends of the input
     “””

     # use a framework that works with quantum circuits, qiskit, cirq, pennylane, etc. 


      # consider print your quantum circuit,


Example:

A = find_negative_numbers([1,-3,2,15])
print(A)

“True”


B = find_negative_numbers([1,4,8,11])
print(B)

“False”


B = find_negative_numbers([-15,-14,2,-1])
print(B)

“True”


## Imports

In [2]:
from math import ceil, log
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, BasicAer, execute
from qiskit.visualization import plot_bloch_multivector, array_to_latex
from qiskit.quantum_info import Statevector
from qiskit.quantum_info.operators import Operator
from qiskit.circuit.library.standard_gates import CXGate, MCXGate
import numpy as np

## Solution Overview 

The challenge for this particular problem is in solving it on a Quantum Computer. On its own, the problem is solved easily by checking for each elements of the array one by one. However, when solving on a quantum computer, there are a couple challenges. 

1. Encoding the elements in the array into the qubits
2. Determining whether an integer, encoded in the qubits, is negative


Each of these challenges and their solutions have been described below the respective functions which resolve them.



### 1. Encoding the Elements  

Encoding the elements in the array implies that corresponding to each element in the array, its binary encoded string should be output equally likely on measurement of the encoding qubits, without any processing on them. 

Here two sets of qubits are used. The first set is used to encode the indices of the array, and the second set is used to encode the actual integer elements of the array. 

Corresponding to each element's bit string a controlled gate is generated which can encode the bit string onto the qubits. The control for the respective gates is set as the bit string encoding the index, which is encoded the first set of registers. 
Each of the index encoding registers is in an equal superposition, hence the probability of an index being selected among the other indices is equal, thus implying that each element in the array has the same probability.

This functionality is implemented by the 'load_inputs' function below

In [3]:
def load_inputs(m, n, vector, vectorbit):  
    a=QuantumRegister(n, 'a')                               # address
    d=QuantumRegister(m, 'd')                               # data
    c = ClassicalRegister(1, 'c')
    qc=QuantumCircuit(a, d, c)
    qc.h(a)                                 # An algorthm to load the given classical data into the ∣d⟩ (data) quibits
    for i in range(len(vector)):
        gate_dict={'0': 'I', '1': 'X'}
        gate_name = '-'.join([gate_dict[b] for b in vectorbit[i]])
        gate_name = '|{}⟩: '.format(vector[i]) + gate_name
        circ = QuantumCircuit(m, name=gate_name)
        for j in range(m):
            if vectorbit[i][j]=='1':
                circ.x(j)
        
        # multi-x gate
        mx_gate = circ.to_gate()

        # multi-polarity multi-control multi-x gate
        mpmcmx_gate = mx_gate.control(n, ctrl_state='{0:0{size}b}'.format(i, size=n)[::-1])
        qc.append(mpmcmx_gate, range(m+n))
    
    qc.measure(d[0], c)

    return qc

### 2. Determine Negative Elements in Encoded Qubits 

To determine whether the integer which has been encoded is negative or not, requires a workaround when encoding the elements. The workaround used here is that if an integer can be encoded in its 2's complement representation, the MSB (Most Significant Bit) can be measured to determine whether the encoded integer is positive or negative. In case of a negative integer, the MSB is always 1, whereas for a positive integer it will be 0. This has been coded into the function itself. 

Inside the function ("find_negative_numbers" below) when it is encoding a number, firstly it determines the number of qubits for encoding the integers (m) by adding 1 to the length of the bitstring for the maximum or minimum value of the array. (In case when negative integers are present, length of the bitstring can be greater than that of other elements in the array). When 2's complement of each integer is computed w.r.t (2**m - 1), the resulting bitstring is always such that there is a 1 in the MSB, if the integer is negative. 

Now, as this is a distinguishing factor, this can be made use of by the function to return whether the array contains a negative integer or not. To do so, we just have to measure the MSB over 1000 shots of the circuit execution, and observe whether the result counts have a '1' or not. If '1' is present, the function returns True, implying the array contains a negative integer , else it returns False.

In [4]:
def find_negative_numbers(vector):
    # n = mimimum number of quibits required to store the addresses of the input
    n=ceil(log(len(vector), 2))

    vector=np.asarray(vector, dtype=int)
    
    # m = minimum number of bits required to store the largest number in the input,
    # m is also the minimum number of qubits required to store all the numbers in the input
    m=max([len('{0:b}'.format(max(vector))), len('{0:b}'.format(np.min(vector)))]) + 1

    vectorbit=[]
    for num in vector:
        
        bin_num = bin(num & (2**m - 1)).replace('0b', '')
        bin_num = '0'*(m - len(bin_num)) + bin_num
        vectorbit.append(bin_num)

    qc = load_inputs(m, n, vector, vectorbit)
    backend = BasicAer.get_backend('qasm_simulator')

    counts = execute(qc, backend, shots= 1000).result().get_counts()
    if '1' in counts.keys():
        return True
    else: 
        return False
    

## Test Cases

In [5]:
# declare test cases
test_cases = []
test_cases.append([10, 5, -20, 63, 22]) #case 1
test_cases.append([1,-3,2,15]) #case 2
test_cases.append([1,4,8,11]) #case 3
test_cases.append([-15,-14,2,-1]) #case 4

In [31]:
for i in test_cases:
    print(find_negative_numbers(i))

True
True
False
True


## Circuit 

The circuit for test case 1 is as below

![Alt text](QOSF_Screening_FindNegativeNumbers_circuit.png)